In [1]:
'''Code for fine-tuning Inception V3 for a new task.

Start with Inception V3 network, not including last fully connected layers.

Train a simple fully connected layer on top of these.


'''

import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
import inception_v3 as inception
'''
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
set_session(tf.Session(config=config))
'''
N_CLASSES = 2
IMSIZE = (299, 299)

# TO DO:: Replace these with paths to the downloaded data.
# Training directory
train_dir = '../data/catdog/train'
# Testing directory
test_dir = '../data/catdog/validation'


# Start with an Inception V3 model, not including the final softmax layer.
base_model = inception.InceptionV3(weights='imagenet')
print ('Loaded Inception model')

Using TensorFlow backend.


Loaded Inception model


In [2]:
import keras
keras.__version__

'1.2.0'

In [3]:
base_model.get_layer('flatten').input

<tf.Tensor 'AvgPool_10:0' shape=(?, 1, 1, 2048) dtype=float32>

In [4]:
# Turn off training on base model layers
for layer in base_model.layers:
    layer.trainable = False
k=base_model.get_layer('flatten').output
# Add on new fully connected layers for the output classes.
x = Dense(32, activation='relu')(base_model.get_layer('flatten').output)
x = Dropout(0.5)(x)
predictions = Dense(N_CLASSES, activation='softmax', name='predictions')(x)

model = Model(input=base_model.input, output=predictions)

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [5]:
#for layer in model.layers:
    #print(layer.output)

In [6]:
# Show some debug output
print (model.summary())

print ('Trainable weights')
#model.save_weights('catdog_pretrain.h5')
#print (model.trainable_weights)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 149, 149, 32)  896         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 149, 149, 32)  128         convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 147, 147, 32)  9248        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [7]:
model.load_weights('catdog_pretrain.h5') 
# Data generators for feeding training/testing images to the model.
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=32,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=32,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        samples_per_epoch=256,
        nb_epoch=0,
        validation_data=test_generator,
        verbose=2,
        nb_val_samples=256)
#model.save_weights('catdog_pretrain.h5')  # always save your weights after training or during training




#img_path = '../data/sport3/validation/hockey/img_2997.jpg'
img_path = '../data/catdog/test/2.jpg'
img = image.load_img(img_path, target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = inception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', preds)
#classes= model.predict_classes(x)
#print('Classes:', classes)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Predicted: [[  8.85941554e-05   9.99911427e-01]]


In [8]:
#model.load_weights('catdog_pretrain.h5') 
#img_path = '../data/sport3/validation/hockey/img_2997.jpg'
img_path = '../data/ta/cat2.jpg'
img = image.load_img(img_path, target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = inception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', preds)
#classes= model.predict_classes(x)
#print('Classes:', classes)

Predicted: [[  9.99395847e-01   6.04182249e-04]]
